In [1]:
import ipynb.fs.full.get_news_headlines as gethl
import ipynb.fs.full.get_article_content as getart
import ipynb.fs.full.write_data as gswd
import ipynb.fs.full.read_data as gsrd 
import ipynb.fs.full.send_mail as mail 
import pandas
import json
import os

In [2]:
def main():
    try:
        hl = gethl.read_livemint_headlines()  #Get headlines
        #outcomes = pandas.DataFrame()
        
        if(not(hl.empty)):
            
            classified_hl = pandas.DataFrame(json.loads(gethl.classify_headlines(hl['title'])))  # converting to json the model reply from Gemini AI.
            classified_titles = hl.join(classified_hl, lsuffix='_orig', rsuffix='_copy')   #make one df given with surety that matching is happening on indexes because Title matching used to give missed matches.. 
            classified_titles = classified_titles[classified_titles.classification]  #keeping only the titles with a analyst recommerndation here.
            classified_titles.drop_duplicates(subset=['link_id'], keep='first', inplace=True, ignore_index=False)        #remove duplicates that can creep in from multiple uls   

            print(f'Found {classified_titles.shape[0]} articles of interest. Proceeding to analyse these articles...')
            
            gswd.write_headlines(classified_titles)  #write titles to a google file

            for idx, row in classified_titles.iterrows():
                print(f'Analysing article {row['link']}...')
                article = getart.get_livemint_articles(row['link'])   #read each article and get it title, subheader, body, and date
                table = getart.get_value_table(article)               # send it to to LLM to make sense of it 
                value_table = pandas.DataFrame(json.loads(table))      # Store the outcomes
                print(f'Found {value_table.shape[0]} recommendations in  article.')
                if(not(value_table.empty)):
                    value_table.insert(0, 'uuid', str(row['uuid']))    # This UUID object is not serializable so better we convert it to string here
                    #value_table.insert(1, 'link', row['link'])        #Just having the UUID should solve the problem  now.
                    #value_table.insert(2, 'site', 'Livemint')
                    #print(value_table)
                    gswd.write_recommendations(value_table)
                else:
                    print(f'No recommendations received for article: {row['link']}')

            recos = gsrd.read_latest_recos()
            mail.send_email(sender_email='me@vishalvivek.com', 
                    recipient_email=['vishalvivek8@gmail.com', 'vishalvivek8@hotmail.com'], 
                    subject=f'Latest recommendations for the day: {recos.iloc[0, 0]}',
                    body = recos
              )
            
            print('Exiting process. Bye!')    
        else:
            print('No Headlines Received')
    except Exception as e:
        print(e)
    return 0

In [3]:
if __name__ == '__main__':
    hl = main()

Executing headline classification for 35 headlines
Found 3 articles of interest. Proceeding to analyse these articles...
Wrote 3 lines to Links sheet in the workbook
Analysing article https://www.livemint.com/market/stock-market-news/stock-market-today-trade-setup-for-nifty-50-to-q4-results-eight-stocks-to-buy-or-sell-on-friday-25-april-2025-11745492290285.html...
Found 8 recommendations in  article.
Wrote 8 lines to Recos  sheet in the workbook
Analysing article https://www.livemint.com/market/stock-market-news/breakout-stocks-to-buy-or-sell-sumeet-bagadia-recommends-five-shares-to-buy-today-25-april-2025-11745517252321.html...
Found 5 recommendations in  article.
Wrote 5 lines to Recos  sheet in the workbook
Analysing article https://www.livemint.com/market/stock-market-news/buy-or-sell-vaishali-parekh-recommends-three-stocks-to-buy-today-25-april-2025-11745544669492.html...
Found 3 recommendations in  article.
Wrote 3 lines to Recos  sheet in the workbook
Email sent successfully!
Ex

In [9]:
recos = gsrd.read_latest_recos()
mail.send_email(sender_email='me@vishalvivek.com', 
                recipient_email=['vishalvivek8@gmail.com', 'vishalvivek8@hotmail.com'], 
                subject=f'Latest recommendations for the day: {recos.iloc[0, 0]}',
                body = recos
              )

Email sent successfully!
